In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn import ensemble, metrics, model_selection, naive_bayes

import lightgbm as lgb
from catboost import CatBoostClassifier
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/root/ml/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X_train = pd.read_csv('train_huge_wv.csv')
X_test = pd.read_csv('test_huge_wv.csv')

In [3]:
Y_train = X_train['Revisit']

In [4]:
# X_train.drop(['Revisit'], axis=1, inplace=True)

In [5]:
X_train_new = X_train[[x for x in X_train.columns if x not in ['Revisit', 'Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']]]
X_test_new = X_test[[x for x in X_test.columns if x not in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']]]

In [6]:
X_train_new = X_train_new.fillna(0.)
X_test_new = X_test_new.fillna(0.)

In [7]:
X_test_new.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
X_train_new.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [8]:
X_train_new = X_train_new.set_index('ID')
X_test_new = X_test_new.set_index('ID')

In [9]:
X_test_new.drop(['Unnamed: 0.1.1'], axis=1, inplace=True)

In [10]:
X_test_new

,Код_диагноза,Возраст,Пол,tfidf_nb_eap_tfidfAnamnes,tfidf_nb_eap_tfidfDiagnoz,tfidf_nb_eap_tfidfGeneral,tfidf_nb_eap_tfidfAllergy,tfidf_nb_eap_tfidfOutside,tfidf_char_nb_eap_Диагноз,tfidf_char_nb_eap_Общее состояние,...,sent_vec_Внешний осмотр_290,sent_vec_Внешний осмотр_291,sent_vec_Внешний осмотр_292,sent_vec_Внешний осмотр_293,sent_vec_Внешний осмотр_294,sent_vec_Внешний осмотр_295,sent_vec_Внешний осмотр_296,sent_vec_Внешний осмотр_297,sent_vec_Внешний осмотр_298,sent_vec_Внешний осмотр_299
ID,,,,,,,,,,,,,,,,,,,,,
0,2725,41,2,0.171361,0.267726,0.000447,0.004574,0.171361,3.194880e-01,1.176980e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1696,64,2,0.171361,0.195164,0.171361,0.038912,0.171361,3.398575e-01,2.339323e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1631,34,2,0.171361,0.323999,0.000095,0.171361,0.171361,8.627880e-01,1.229226e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1088,58,2,0.171361,0.102255,0.171361,0.171361,0.000003,3.051018e-02,2.339323e-01,...,0.007669,0.045423,0.013931,0.031605,-0.039387,-0.025188,-0.003363,-0.044734,-0.015943,-0.067938
4,1581,34,2,0.041676,0.854828,0.171361,0.171361,0.171361,9.992496e-01,2.339323e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1692,51,1,0.016239,0.436443,0.171361,0.171361,0.171361,6.645557e-01,2.339323e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2005,25,2,0.171361,0.000253,0.000447,0.004730,0.171361,8.997162e-07,1.176980e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1375,30,2,0.004112,0.229043,0.000447,0.171361,0.171361,1.967807e-01,1.176980e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1681,63,2,0.171361,0.901874,0.008878,0.038912,0.171361,9.939881e-01,5.970388e-05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
xttrain, xttest, yttrain, yttest = train_test_split(X_train_new, Y_train, test_size=0.2, random_state=42)

In [17]:
cclf = CatBoostClassifier(n_estimators=1000, learning_rate=0.03, eval_metric='AUC', depth=7)
cclf.fit(xttrain, yttrain)
cclf.score(xttest, yttest)

0:	learn: 0.7067715	total: 683ms	remaining: 11m 22s
1:	learn: 0.7111071	total: 1.4s	remaining: 11m 38s
2:	learn: 0.7140366	total: 2.11s	remaining: 11m 40s
3:	learn: 0.7138860	total: 2.82s	remaining: 11m 43s
4:	learn: 0.7155113	total: 3.54s	remaining: 11m 43s
5:	learn: 0.7157955	total: 4.24s	remaining: 11m 43s
6:	learn: 0.7157985	total: 4.95s	remaining: 11m 42s
7:	learn: 0.7159395	total: 5.69s	remaining: 11m 45s
8:	learn: 0.7163688	total: 6.4s	remaining: 11m 44s
9:	learn: 0.7166857	total: 7.11s	remaining: 11m 43s
10:	learn: 0.7169919	total: 7.74s	remaining: 11m 36s
11:	learn: 0.7170071	total: 8.36s	remaining: 11m 28s
12:	learn: 0.7175698	total: 9.11s	remaining: 11m 31s
13:	learn: 0.7179979	total: 9.85s	remaining: 11m 33s
14:	learn: 0.7181075	total: 10.6s	remaining: 11m 34s
15:	learn: 0.7180283	total: 11.3s	remaining: 11m 34s
16:	learn: 0.7181981	total: 12s	remaining: 11m 34s
17:	learn: 0.7184306	total: 12.6s	remaining: 11m 29s
18:	learn: 0.7184872	total: 13.3s	remaining: 11m 24s
19:	lea

155:	learn: 0.7282418	total: 1m 49s	remaining: 9m 53s
156:	learn: 0.7282742	total: 1m 50s	remaining: 9m 52s
157:	learn: 0.7283427	total: 1m 51s	remaining: 9m 52s
158:	learn: 0.7284151	total: 1m 51s	remaining: 9m 51s
159:	learn: 0.7284967	total: 1m 52s	remaining: 9m 50s
160:	learn: 0.7285786	total: 1m 53s	remaining: 9m 50s
161:	learn: 0.7286538	total: 1m 53s	remaining: 9m 49s
162:	learn: 0.7287178	total: 1m 54s	remaining: 9m 49s
163:	learn: 0.7287836	total: 1m 55s	remaining: 9m 48s
164:	learn: 0.7288672	total: 1m 56s	remaining: 9m 47s
165:	learn: 0.7289424	total: 1m 56s	remaining: 9m 46s
166:	learn: 0.7289761	total: 1m 57s	remaining: 9m 46s
167:	learn: 0.7290367	total: 1m 58s	remaining: 9m 45s
168:	learn: 0.7290728	total: 1m 58s	remaining: 9m 45s
169:	learn: 0.7291557	total: 1m 59s	remaining: 9m 44s
170:	learn: 0.7292049	total: 2m	remaining: 9m 43s
171:	learn: 0.7292534	total: 2m 1s	remaining: 9m 42s
172:	learn: 0.7292988	total: 2m 1s	remaining: 9m 41s
173:	learn: 0.7293460	total: 2m 2s

308:	learn: 0.7375423	total: 3m 37s	remaining: 8m 6s
309:	learn: 0.7375911	total: 3m 38s	remaining: 8m 6s
310:	learn: 0.7376521	total: 3m 39s	remaining: 8m 5s
311:	learn: 0.7377193	total: 3m 39s	remaining: 8m 4s
312:	learn: 0.7378017	total: 3m 40s	remaining: 8m 3s
313:	learn: 0.7378467	total: 3m 41s	remaining: 8m 3s
314:	learn: 0.7379088	total: 3m 41s	remaining: 8m 2s
315:	learn: 0.7379437	total: 3m 42s	remaining: 8m 1s
316:	learn: 0.7380161	total: 3m 43s	remaining: 8m 1s
317:	learn: 0.7380824	total: 3m 44s	remaining: 8m
318:	learn: 0.7381232	total: 3m 44s	remaining: 8m
319:	learn: 0.7381880	total: 3m 45s	remaining: 7m 59s
320:	learn: 0.7382373	total: 3m 46s	remaining: 7m 58s
321:	learn: 0.7383008	total: 3m 46s	remaining: 7m 57s
322:	learn: 0.7383728	total: 3m 47s	remaining: 7m 57s
323:	learn: 0.7384189	total: 3m 48s	remaining: 7m 56s
324:	learn: 0.7384726	total: 3m 49s	remaining: 7m 55s
325:	learn: 0.7385296	total: 3m 49s	remaining: 7m 55s
326:	learn: 0.7385593	total: 3m 50s	remaining

462:	learn: 0.7460253	total: 5m 26s	remaining: 6m 18s
463:	learn: 0.7460681	total: 5m 26s	remaining: 6m 17s
464:	learn: 0.7461365	total: 5m 27s	remaining: 6m 16s
465:	learn: 0.7462049	total: 5m 28s	remaining: 6m 16s
466:	learn: 0.7462501	total: 5m 28s	remaining: 6m 15s
467:	learn: 0.7463097	total: 5m 29s	remaining: 6m 14s
468:	learn: 0.7463722	total: 5m 30s	remaining: 6m 13s
469:	learn: 0.7464347	total: 5m 30s	remaining: 6m 13s
470:	learn: 0.7464822	total: 5m 31s	remaining: 6m 12s
471:	learn: 0.7465487	total: 5m 32s	remaining: 6m 11s
472:	learn: 0.7466378	total: 5m 33s	remaining: 6m 11s
473:	learn: 0.7466754	total: 5m 33s	remaining: 6m 10s
474:	learn: 0.7467342	total: 5m 34s	remaining: 6m 9s
475:	learn: 0.7467877	total: 5m 35s	remaining: 6m 9s
476:	learn: 0.7468338	total: 5m 36s	remaining: 6m 8s
477:	learn: 0.7468909	total: 5m 36s	remaining: 6m 7s
478:	learn: 0.7469588	total: 5m 37s	remaining: 6m 7s
479:	learn: 0.7469852	total: 5m 38s	remaining: 6m 6s
480:	learn: 0.7470146	total: 5m 38

616:	learn: 0.7536497	total: 7m 14s	remaining: 4m 29s
617:	learn: 0.7537095	total: 7m 15s	remaining: 4m 29s
618:	learn: 0.7537742	total: 7m 16s	remaining: 4m 28s
619:	learn: 0.7538096	total: 7m 16s	remaining: 4m 27s
620:	learn: 0.7538573	total: 7m 17s	remaining: 4m 27s
621:	learn: 0.7539069	total: 7m 18s	remaining: 4m 26s
622:	learn: 0.7539621	total: 7m 18s	remaining: 4m 25s
623:	learn: 0.7540219	total: 7m 19s	remaining: 4m 24s
624:	learn: 0.7540726	total: 7m 20s	remaining: 4m 24s
625:	learn: 0.7541321	total: 7m 21s	remaining: 4m 23s
626:	learn: 0.7541790	total: 7m 21s	remaining: 4m 22s
627:	learn: 0.7542171	total: 7m 22s	remaining: 4m 22s
628:	learn: 0.7542672	total: 7m 23s	remaining: 4m 21s
629:	learn: 0.7543198	total: 7m 23s	remaining: 4m 20s
630:	learn: 0.7543504	total: 7m 24s	remaining: 4m 19s
631:	learn: 0.7543959	total: 7m 25s	remaining: 4m 19s
632:	learn: 0.7544534	total: 7m 25s	remaining: 4m 18s
633:	learn: 0.7545193	total: 7m 26s	remaining: 4m 17s
634:	learn: 0.7545873	total:

769:	learn: 0.7611217	total: 9m 2s	remaining: 2m 42s
770:	learn: 0.7611652	total: 9m 3s	remaining: 2m 41s
771:	learn: 0.7612355	total: 9m 4s	remaining: 2m 40s
772:	learn: 0.7612929	total: 9m 4s	remaining: 2m 40s
773:	learn: 0.7613521	total: 9m 5s	remaining: 2m 39s
774:	learn: 0.7614107	total: 9m 6s	remaining: 2m 38s
775:	learn: 0.7614598	total: 9m 6s	remaining: 2m 37s
776:	learn: 0.7614836	total: 9m 7s	remaining: 2m 37s
777:	learn: 0.7615373	total: 9m 8s	remaining: 2m 36s
778:	learn: 0.7615900	total: 9m 9s	remaining: 2m 35s
779:	learn: 0.7616425	total: 9m 9s	remaining: 2m 35s
780:	learn: 0.7616654	total: 9m 10s	remaining: 2m 34s
781:	learn: 0.7617326	total: 9m 11s	remaining: 2m 33s
782:	learn: 0.7617798	total: 9m 11s	remaining: 2m 32s
783:	learn: 0.7618427	total: 9m 12s	remaining: 2m 32s
784:	learn: 0.7618929	total: 9m 13s	remaining: 2m 31s
785:	learn: 0.7619468	total: 9m 14s	remaining: 2m 30s
786:	learn: 0.7620189	total: 9m 14s	remaining: 2m 30s
787:	learn: 0.7620738	total: 9m 15s	rem

921:	learn: 0.7683802	total: 10m 49s	remaining: 54.9s
922:	learn: 0.7683971	total: 10m 49s	remaining: 54.2s
923:	learn: 0.7684480	total: 10m 50s	remaining: 53.5s
924:	learn: 0.7684851	total: 10m 51s	remaining: 52.8s
925:	learn: 0.7685109	total: 10m 51s	remaining: 52.1s
926:	learn: 0.7685387	total: 10m 52s	remaining: 51.4s
927:	learn: 0.7685768	total: 10m 53s	remaining: 50.7s
928:	learn: 0.7686168	total: 10m 54s	remaining: 50s
929:	learn: 0.7686757	total: 10m 54s	remaining: 49.3s
930:	learn: 0.7687063	total: 10m 55s	remaining: 48.6s
931:	learn: 0.7687644	total: 10m 56s	remaining: 47.9s
932:	learn: 0.7688276	total: 10m 56s	remaining: 47.2s
933:	learn: 0.7688634	total: 10m 57s	remaining: 46.5s
934:	learn: 0.7689253	total: 10m 58s	remaining: 45.7s
935:	learn: 0.7689764	total: 10m 58s	remaining: 45s
936:	learn: 0.7690106	total: 10m 59s	remaining: 44.3s
937:	learn: 0.7690626	total: 11m	remaining: 43.6s
938:	learn: 0.7691073	total: 11m	remaining: 42.9s
939:	learn: 0.7691425	total: 11m 1s	rema

0.8283584725705606

In [41]:
Submit_v1 = pd.DataFrame(data={'proba': cclf.predict_proba(X_test_new)[:,1].ravel()}, index=X_test_new.index)

In [42]:
Submit_v1.to_csv('v1.csv')

In [20]:
rf_model = RandomForestClassifier(
    n_jobs=24,
    class_weight={0: 1, 1: 4.8},
    n_estimators=500,
    max_depth=5
)

rf_model.fit(xttrain, yttrain)
rf_model.score(xttest, yttest)

0.8279875657917977

In [24]:
rf_model2 = RandomForestClassifier(
    n_jobs=24,
    class_weight={0: 1, 1: 4.8},
    n_estimators=500,
    max_depth=15
)

rf_model2.fit(xttrain, yttrain)
metrics.roc_auc_score(rf_model2.predict(xttest), yttest), rf_model2.score(xttest, yttest)

(0.6690480176726055, 0.8280052280193578)

In [58]:
lgbm_model = lgb.LGBMClassifier(
    max_depth=20,
    n_estimators=10,
    learning_rate=0.01,
    num_leaves=16,
    class_weight={0:1, 1:4.8},
    reg_lambda=0.1, silent=False
)

lgbm_model.fit(xttrain, yttrain)
metrics.roc_auc_score(lgbm_model.predict(xttest), yttest)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [63]:
import catboost._catboost

AttributeError: type object '_catboost._FloatArrayWrapper' has no attribute '__reduce_cython__'

In [90]:
# X_test_new

In [15]:
vclf = VotingClassifier(
    estimators=[
        ('cat1', CatBoostClassifier(n_estimators=500, 
                                    learning_rate=0.02,
                                    eval_metric='AUC', 
                                    depth=7)),
        
        ('cat3', CatBoostClassifier(n_estimators=1000,
                          learning_rate=0.01,
                          l2_leaf_reg=0.1,
                          eval_metric='AUC', 
                          depth=5,
                          rsm=0.8,
                          logging_level='Info',
                          class_weights=[1, 4.8]))
    ], 
    voting='soft'
)

vclf.fit(xttrain, yttrain)
metrics.roc_auc_score(vclf.predict(xttest), yttest)

# scores = cross_val_score(estimator=vclf, X=X_train_new, y=Y_train, cv=3, scoring='accuracy')

0:	learn: 0.7082807	total: 711ms	remaining: 5m 54s
1:	learn: 0.7108582	total: 1.33s	remaining: 5m 30s
2:	learn: 0.7106073	total: 1.97s	remaining: 5m 26s
3:	learn: 0.7120623	total: 2.63s	remaining: 5m 25s
4:	learn: 0.7122703	total: 3.27s	remaining: 5m 23s
5:	learn: 0.7119297	total: 4s	remaining: 5m 29s
6:	learn: 0.7127200	total: 4.67s	remaining: 5m 28s
7:	learn: 0.7138436	total: 5.39s	remaining: 5m 31s
8:	learn: 0.7143996	total: 6.13s	remaining: 5m 34s
9:	learn: 0.7147626	total: 6.88s	remaining: 5m 37s
10:	learn: 0.7150464	total: 7.6s	remaining: 5m 37s
11:	learn: 0.7151794	total: 8.35s	remaining: 5m 39s
12:	learn: 0.7153203	total: 9.1s	remaining: 5m 40s
13:	learn: 0.7155057	total: 9.84s	remaining: 5m 41s
14:	learn: 0.7155477	total: 10.6s	remaining: 5m 41s
15:	learn: 0.7159933	total: 11.2s	remaining: 5m 39s
16:	learn: 0.7162327	total: 11.9s	remaining: 5m 38s
17:	learn: 0.7166206	total: 12.6s	remaining: 5m 37s
18:	learn: 0.7167518	total: 13.3s	remaining: 5m 37s
19:	learn: 0.7167463	total:

157:	learn: 0.7249224	total: 1m 59s	remaining: 4m 17s
158:	learn: 0.7249786	total: 1m 59s	remaining: 4m 17s
159:	learn: 0.7250180	total: 2m	remaining: 4m 16s
160:	learn: 0.7250825	total: 2m 1s	remaining: 4m 15s
161:	learn: 0.7251642	total: 2m 2s	remaining: 4m 14s
162:	learn: 0.7252062	total: 2m 2s	remaining: 4m 14s
163:	learn: 0.7252511	total: 2m 3s	remaining: 4m 13s
164:	learn: 0.7252673	total: 2m 4s	remaining: 4m 12s
165:	learn: 0.7253185	total: 2m 5s	remaining: 4m 12s
166:	learn: 0.7253508	total: 2m 6s	remaining: 4m 11s
167:	learn: 0.7253949	total: 2m 6s	remaining: 4m 10s
168:	learn: 0.7254391	total: 2m 7s	remaining: 4m 9s
169:	learn: 0.7254867	total: 2m 8s	remaining: 4m 8s
170:	learn: 0.7255558	total: 2m 8s	remaining: 4m 8s
171:	learn: 0.7256035	total: 2m 9s	remaining: 4m 7s
172:	learn: 0.7256698	total: 2m 10s	remaining: 4m 6s
173:	learn: 0.7257223	total: 2m 11s	remaining: 4m 5s
174:	learn: 0.7257637	total: 2m 11s	remaining: 4m 5s
175:	learn: 0.7258072	total: 2m 12s	remaining: 4m 4

310:	learn: 0.7314561	total: 5m 43s	remaining: 3m 28s
311:	learn: 0.7315024	total: 5m 44s	remaining: 3m 27s
312:	learn: 0.7315638	total: 5m 46s	remaining: 3m 27s
313:	learn: 0.7316110	total: 5m 48s	remaining: 3m 26s
314:	learn: 0.7316438	total: 5m 51s	remaining: 3m 26s
315:	learn: 0.7316961	total: 5m 52s	remaining: 3m 25s
316:	learn: 0.7317469	total: 5m 54s	remaining: 3m 24s
317:	learn: 0.7317976	total: 5m 55s	remaining: 3m 23s
318:	learn: 0.7318359	total: 5m 57s	remaining: 3m 23s
319:	learn: 0.7318672	total: 5m 58s	remaining: 3m 21s
320:	learn: 0.7319193	total: 5m 59s	remaining: 3m 20s
321:	learn: 0.7319738	total: 6m 1s	remaining: 3m 19s
322:	learn: 0.7320227	total: 6m 4s	remaining: 3m 19s
323:	learn: 0.7320636	total: 6m 5s	remaining: 3m 18s
324:	learn: 0.7320895	total: 6m 7s	remaining: 3m 17s
325:	learn: 0.7321278	total: 6m 9s	remaining: 3m 17s
326:	learn: 0.7321603	total: 6m 11s	remaining: 3m 16s
327:	learn: 0.7321837	total: 6m 13s	remaining: 3m 16s
328:	learn: 0.7322285	total: 6m 1

464:	learn: 0.7376023	total: 9m 3s	remaining: 40.9s
465:	learn: 0.7376357	total: 9m 4s	remaining: 39.7s
466:	learn: 0.7376911	total: 9m 5s	remaining: 38.5s
467:	learn: 0.7377158	total: 9m 6s	remaining: 37.3s
468:	learn: 0.7377621	total: 9m 6s	remaining: 36.1s
469:	learn: 0.7378004	total: 9m 7s	remaining: 34.9s
470:	learn: 0.7378376	total: 9m 8s	remaining: 33.7s
471:	learn: 0.7378640	total: 9m 8s	remaining: 32.6s
472:	learn: 0.7379208	total: 9m 9s	remaining: 31.4s
473:	learn: 0.7379729	total: 9m 10s	remaining: 30.2s
474:	learn: 0.7380159	total: 9m 11s	remaining: 29s
475:	learn: 0.7380645	total: 9m 11s	remaining: 27.8s
476:	learn: 0.7380928	total: 9m 12s	remaining: 26.6s
477:	learn: 0.7381138	total: 9m 13s	remaining: 25.5s
478:	learn: 0.7381442	total: 9m 13s	remaining: 24.3s
479:	learn: 0.7381839	total: 9m 14s	remaining: 23.1s
480:	learn: 0.7382252	total: 9m 15s	remaining: 21.9s
481:	learn: 0.7382535	total: 9m 16s	remaining: 20.8s
482:	learn: 0.7382992	total: 9m 16s	remaining: 19.6s
483:

tfidf_nb_eap_tfidfDiagnoz, bin=3 score 1505.612169
21:	learn: 0.7102264	total: 10.2s	remaining: 36.1s

sent_vec_Анамнез заболевания_272, bin=0 score 1474.106164
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 1473.776716
sent_vec_Общее состояние_162, bin=122 score 1480.199672
sent_vec_аллергия_40, bin=54 score 1479.209189
sent_vec_Диагноз_168, bin=49 score 1477.299622
22:	learn: 0.7104370	total: 10.6s	remaining: 35.6s

sent_vec_Анамнез заболевания_37, bin=127 score 1450.131126
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 1450.268602
sent_vec_Общее состояние_270, bin=4 score 1452.380426
Возраст, bin=19 score 1448.849267
sent_vec_Диагноз_96, bin=34 score 1450.444534
23:	learn: 0.7105215	total: 11.1s	remaining: 35s

sent_vec_Анамнез заболевания_272, bin=0 score 1422.211126
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 1419.171918
sent_vec_Общее состояние_268, bin=5 score 1421.752695
Возраст, bin=21 score 1423.416063
tfidf_nb_eap_tfidfDiagnoz, bin=1 score 1424.180417
24:	learn: 0.7105460	total: 11.5s	remaini

48:	learn: 0.7104440	total: 22.6s	remaining: 23.6s

sent_vec_Анамнез заболевания_272, bin=0 score 926.9911982
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 931.32237
sent_vec_Общее состояние_162, bin=122 score 932.9987753
Возраст, bin=18 score 934.8354168
Код_диагноза_1, bin=44 score 933.8133816
49:	learn: 0.7105436	total: 23.1s	remaining: 23.1s

sent_vec_Анамнез заболевания_37, bin=127 score 911.4691442
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 916.021495
sent_vec_Общее состояние_236, bin=123 score 920.6650516
Возраст, bin=18 score 917.3664125
count_words_nb_eap_Диагноз, bin=95 score 918.5325112
50:	learn: 0.7104276	total: 23.6s	remaining: 22.7s

sent_vec_Анамнез заболевания_272, bin=0 score 897.4516713
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 903.2610033
sent_vec_Общее состояние_162, bin=122 score 905.1991195
Возраст, bin=18 score 904.5263254
tfidf_char_nb_eap_Диагноз, bin=127 score 905.0302577
51:	learn: 0.7104030	total: 24s	remaining: 22.2s

sent_vec_Анамнез заболевания_272, bin=0 score 883

sent_vec_Общее состояние_162, bin=122 score 642.4898536
Возраст, bin=18 score 640.4306468
sent_vec_Диагноз_64, bin=97 score 640.223925
76:	learn: 0.7103649	total: 35.7s	remaining: 10.7s

sent_vec_Анамнез заболевания_37, bin=127 score 622.9753981
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 630.5645887
sent_vec_Общее состояние_162, bin=122 score 633.386998
Возраст, bin=18 score 634.3564616
sent_vec_аллергия_141, bin=89 score 635.7898449
77:	learn: 0.7103743	total: 36.1s	remaining: 10.2s

sent_vec_Анамнез заболевания_37, bin=127 score 613.182971
count_words_nb_eap_Диагноз, bin=69 score 621.5697849
sent_vec_Общее состояние_136, bin=3 score 624.0515939
Возраст, bin=18 score 625.2418388
sent_vec_Диагноз_216, bin=0 score 623.7303417
78:	learn: 0.7103711	total: 36.6s	remaining: 9.72s

sent_vec_Анамнез заболевания_37, bin=127 score 605.9827114
tfidf_nb_eap_tfidfDiagnoz, bin=57 score 614.8517277
sent_vec_Общее состояние_33, bin=123 score 617.1606164
Возраст, bin=18 score 618.4572343
sent_vec_Диагноз

0.9147222467543937

In [14]:
cclf3 = CatBoostClassifier(n_estimators=10,
                          learning_rate=0.01,
                          l2_leaf_reg=0.1,
                          eval_metric='AUC', 
                          depth=16,
                          rsm=0.8,
                          logging_level='Info',
                          class_weights=[1, 4.8])
cclf3.fit(xttrain, yttrain)
cclf3.score(xttest, yttest)

NameError: name 'PY3' is not defined

In [101]:
scores = cross_val_score(estimator=cclf, X=X_train_new, y=Y_train, cv=5, scoring='accuracy')

0:	learn: 0.7069215	total: 152ms	remaining: 2m 31s
1:	learn: 0.7085329	total: 291ms	remaining: 2m 25s
2:	learn: 0.7102051	total: 432ms	remaining: 2m 23s
3:	learn: 0.7109856	total: 560ms	remaining: 2m 19s
4:	learn: 0.7116216	total: 685ms	remaining: 2m 16s
5:	learn: 0.7114200	total: 795ms	remaining: 2m 11s
6:	learn: 0.7125956	total: 915ms	remaining: 2m 9s
7:	learn: 0.7130060	total: 1.03s	remaining: 2m 8s
8:	learn: 0.7130644	total: 1.15s	remaining: 2m 6s
9:	learn: 0.7136109	total: 1.26s	remaining: 2m 5s
10:	learn: 0.7141126	total: 1.38s	remaining: 2m 4s
11:	learn: 0.7142845	total: 1.5s	remaining: 2m 3s
12:	learn: 0.7144773	total: 1.62s	remaining: 2m 3s
13:	learn: 0.7146797	total: 1.75s	remaining: 2m 3s
14:	learn: 0.7147170	total: 1.87s	remaining: 2m 2s
15:	learn: 0.7147757	total: 1.98s	remaining: 2m 2s
16:	learn: 0.7149208	total: 2.1s	remaining: 2m 1s
17:	learn: 0.7151095	total: 2.22s	remaining: 2m 1s
18:	learn: 0.7152581	total: 2.35s	remaining: 2m 1s
19:	learn: 0.7153415	total: 2.47s	rem

162:	learn: 0.7226194	total: 22.3s	remaining: 1m 54s
163:	learn: 0.7226737	total: 22.4s	remaining: 1m 54s
164:	learn: 0.7227041	total: 22.5s	remaining: 1m 54s
165:	learn: 0.7227506	total: 22.7s	remaining: 1m 53s
166:	learn: 0.7227755	total: 22.8s	remaining: 1m 53s
167:	learn: 0.7227945	total: 23s	remaining: 1m 53s
168:	learn: 0.7228368	total: 23.1s	remaining: 1m 53s
169:	learn: 0.7228606	total: 23.2s	remaining: 1m 53s
170:	learn: 0.7228911	total: 23.4s	remaining: 1m 53s
171:	learn: 0.7229965	total: 23.5s	remaining: 1m 53s
172:	learn: 0.7230175	total: 23.6s	remaining: 1m 53s
173:	learn: 0.7230860	total: 23.8s	remaining: 1m 52s
174:	learn: 0.7231214	total: 23.9s	remaining: 1m 52s
175:	learn: 0.7231490	total: 24.1s	remaining: 1m 52s
176:	learn: 0.7231782	total: 24.2s	remaining: 1m 52s
177:	learn: 0.7232465	total: 24.3s	remaining: 1m 52s
178:	learn: 0.7232666	total: 24.5s	remaining: 1m 52s
179:	learn: 0.7232855	total: 24.6s	remaining: 1m 52s
180:	learn: 0.7233316	total: 24.8s	remaining: 1m

318:	learn: 0.7285483	total: 44.4s	remaining: 1m 34s
319:	learn: 0.7285860	total: 44.6s	remaining: 1m 34s
320:	learn: 0.7286045	total: 44.7s	remaining: 1m 34s
321:	learn: 0.7286433	total: 44.8s	remaining: 1m 34s
322:	learn: 0.7286701	total: 45s	remaining: 1m 34s
323:	learn: 0.7286910	total: 45.1s	remaining: 1m 34s
324:	learn: 0.7287121	total: 45.2s	remaining: 1m 33s
325:	learn: 0.7287331	total: 45.4s	remaining: 1m 33s
326:	learn: 0.7287488	total: 45.6s	remaining: 1m 33s
327:	learn: 0.7287787	total: 45.7s	remaining: 1m 33s
328:	learn: 0.7288128	total: 45.9s	remaining: 1m 33s
329:	learn: 0.7288466	total: 46s	remaining: 1m 33s
330:	learn: 0.7288908	total: 46.2s	remaining: 1m 33s
331:	learn: 0.7289585	total: 46.3s	remaining: 1m 33s
332:	learn: 0.7290306	total: 46.5s	remaining: 1m 33s
333:	learn: 0.7290463	total: 46.6s	remaining: 1m 32s
334:	learn: 0.7290784	total: 46.7s	remaining: 1m 32s
335:	learn: 0.7291300	total: 46.9s	remaining: 1m 32s
336:	learn: 0.7291453	total: 47s	remaining: 1m 32s

475:	learn: 0.7336950	total: 1m 6s	remaining: 1m 13s
476:	learn: 0.7337240	total: 1m 6s	remaining: 1m 13s
477:	learn: 0.7337527	total: 1m 6s	remaining: 1m 12s
478:	learn: 0.7337766	total: 1m 6s	remaining: 1m 12s
479:	learn: 0.7338052	total: 1m 7s	remaining: 1m 12s
480:	learn: 0.7338240	total: 1m 7s	remaining: 1m 12s
481:	learn: 0.7338699	total: 1m 7s	remaining: 1m 12s
482:	learn: 0.7338974	total: 1m 7s	remaining: 1m 12s
483:	learn: 0.7339395	total: 1m 7s	remaining: 1m 12s
484:	learn: 0.7339968	total: 1m 7s	remaining: 1m 12s
485:	learn: 0.7340353	total: 1m 7s	remaining: 1m 11s
486:	learn: 0.7340613	total: 1m 8s	remaining: 1m 11s
487:	learn: 0.7340806	total: 1m 8s	remaining: 1m 11s
488:	learn: 0.7340904	total: 1m 8s	remaining: 1m 11s
489:	learn: 0.7341173	total: 1m 8s	remaining: 1m 11s
490:	learn: 0.7341399	total: 1m 8s	remaining: 1m 11s
491:	learn: 0.7341661	total: 1m 8s	remaining: 1m 11s
492:	learn: 0.7341873	total: 1m 8s	remaining: 1m 10s
493:	learn: 0.7342036	total: 1m 9s	remaining: 

631:	learn: 0.7381944	total: 1m 28s	remaining: 51.5s
632:	learn: 0.7382275	total: 1m 28s	remaining: 51.3s
633:	learn: 0.7382441	total: 1m 28s	remaining: 51.2s
634:	learn: 0.7382894	total: 1m 28s	remaining: 51.1s
635:	learn: 0.7383180	total: 1m 28s	remaining: 50.9s
636:	learn: 0.7383414	total: 1m 29s	remaining: 50.8s
637:	learn: 0.7383724	total: 1m 29s	remaining: 50.6s
638:	learn: 0.7383951	total: 1m 29s	remaining: 50.5s
639:	learn: 0.7384404	total: 1m 29s	remaining: 50.4s
640:	learn: 0.7384574	total: 1m 29s	remaining: 50.2s
641:	learn: 0.7385094	total: 1m 29s	remaining: 50.1s
642:	learn: 0.7385307	total: 1m 29s	remaining: 50s
643:	learn: 0.7385650	total: 1m 30s	remaining: 49.8s
644:	learn: 0.7386129	total: 1m 30s	remaining: 49.7s
645:	learn: 0.7386343	total: 1m 30s	remaining: 49.6s
646:	learn: 0.7386696	total: 1m 30s	remaining: 49.4s
647:	learn: 0.7386800	total: 1m 30s	remaining: 49.3s
648:	learn: 0.7387056	total: 1m 30s	remaining: 49.2s
649:	learn: 0.7387234	total: 1m 31s	remaining: 4

...

240:	learn: 0.7257698	total: 36s	remaining: 1m 53s
241:	learn: 0.7257933	total: 36.2s	remaining: 1m 53s
242:	learn: 0.7258154	total: 36.3s	remaining: 1m 53s
243:	learn: 0.7258624	total: 36.5s	remaining: 1m 52s
244:	learn: 0.7259069	total: 36.6s	remaining: 1m 52s
245:	learn: 0.7259580	total: 36.8s	remaining: 1m 52s
246:	learn: 0.7260128	total: 36.9s	remaining: 1m 52s
247:	learn: 0.7260359	total: 37.1s	remaining: 1m 52s
248:	learn: 0.7260523	total: 37.2s	remaining: 1m 52s
249:	learn: 0.7260904	total: 37.4s	remaining: 1m 52s
250:	learn: 0.7261235	total: 37.5s	remaining: 1m 51s
251:	learn: 0.7261865	total: 37.7s	remaining: 1m 51s
252:	learn: 0.7262241	total: 37.8s	remaining: 1m 51s
253:	learn: 0.7262658	total: 37.9s	remaining: 1m 51s
254:	learn: 0.7263115	total: 38.1s	remaining: 1m 51s
255:	learn: 0.7263479	total: 38.2s	remaining: 1m 51s
256:	learn: 0.7263956	total: 38.4s	remaining: 1m 50s
257:	learn: 0.7264284	total: 38.5s	remaining: 1m 50s
258:	learn: 0.7264836	total: 38.7s	remaining: 1m

396:	learn: 0.7310173	total: 1m	remaining: 1m 31s
397:	learn: 0.7310447	total: 1m	remaining: 1m 31s
398:	learn: 0.7310843	total: 1m	remaining: 1m 31s
399:	learn: 0.7311299	total: 1m	remaining: 1m 30s
400:	learn: 0.7311712	total: 1m	remaining: 1m 30s
401:	learn: 0.7312348	total: 1m	remaining: 1m 30s
402:	learn: 0.7312643	total: 1m 1s	remaining: 1m 30s
403:	learn: 0.7312842	total: 1m 1s	remaining: 1m 30s
404:	learn: 0.7313356	total: 1m 1s	remaining: 1m 30s
405:	learn: 0.7313777	total: 1m 1s	remaining: 1m 30s
406:	learn: 0.7314344	total: 1m 1s	remaining: 1m 29s
407:	learn: 0.7314538	total: 1m 1s	remaining: 1m 29s
408:	learn: 0.7314707	total: 1m 2s	remaining: 1m 29s
409:	learn: 0.7314912	total: 1m 2s	remaining: 1m 29s
410:	learn: 0.7315243	total: 1m 2s	remaining: 1m 29s
411:	learn: 0.7315615	total: 1m 2s	remaining: 1m 29s
412:	learn: 0.7316166	total: 1m 2s	remaining: 1m 29s
413:	learn: 0.7316466	total: 1m 2s	remaining: 1m 29s
414:	learn: 0.7316776	total: 1m 3s	remaining: 1m 28s
415:	learn:

550:	learn: 0.7359020	total: 1m 23s	remaining: 1m 8s
551:	learn: 0.7359505	total: 1m 23s	remaining: 1m 8s
552:	learn: 0.7359757	total: 1m 24s	remaining: 1m 7s
553:	learn: 0.7360003	total: 1m 24s	remaining: 1m 7s
554:	learn: 0.7360141	total: 1m 24s	remaining: 1m 7s
555:	learn: 0.7360425	total: 1m 24s	remaining: 1m 7s
556:	learn: 0.7360616	total: 1m 24s	remaining: 1m 7s
557:	learn: 0.7361156	total: 1m 24s	remaining: 1m 7s
558:	learn: 0.7361778	total: 1m 25s	remaining: 1m 7s
559:	learn: 0.7362056	total: 1m 25s	remaining: 1m 6s
560:	learn: 0.7362395	total: 1m 25s	remaining: 1m 6s
561:	learn: 0.7362679	total: 1m 25s	remaining: 1m 6s
562:	learn: 0.7362882	total: 1m 25s	remaining: 1m 6s
563:	learn: 0.7363114	total: 1m 25s	remaining: 1m 6s
564:	learn: 0.7363474	total: 1m 25s	remaining: 1m 6s
565:	learn: 0.7363878	total: 1m 26s	remaining: 1m 6s
566:	learn: 0.7364085	total: 1m 26s	remaining: 1m 5s
567:	learn: 0.7364272	total: 1m 26s	remaining: 1m 5s
568:	learn: 0.7364541	total: 1m 26s	remaining:

706:	learn: 0.7404487	total: 1m 47s	remaining: 44.6s
707:	learn: 0.7404772	total: 1m 47s	remaining: 44.5s
708:	learn: 0.7405120	total: 1m 47s	remaining: 44.3s
709:	learn: 0.7405274	total: 1m 48s	remaining: 44.2s
710:	learn: 0.7405410	total: 1m 48s	remaining: 44s
711:	learn: 0.7405631	total: 1m 48s	remaining: 43.9s
712:	learn: 0.7405855	total: 1m 48s	remaining: 43.7s
713:	learn: 0.7406123	total: 1m 48s	remaining: 43.6s
714:	learn: 0.7406325	total: 1m 48s	remaining: 43.4s
715:	learn: 0.7406584	total: 1m 49s	remaining: 43.3s
716:	learn: 0.7406786	total: 1m 49s	remaining: 43.1s
717:	learn: 0.7407015	total: 1m 49s	remaining: 42.9s
718:	learn: 0.7407315	total: 1m 49s	remaining: 42.8s
719:	learn: 0.7407631	total: 1m 49s	remaining: 42.6s
720:	learn: 0.7407818	total: 1m 49s	remaining: 42.5s
721:	learn: 0.7408022	total: 1m 49s	remaining: 42.3s
722:	learn: 0.7408196	total: 1m 50s	remaining: 42.2s
723:	learn: 0.7408466	total: 1m 50s	remaining: 42s
724:	learn: 0.7408583	total: 1m 50s	remaining: 41.

864:	learn: 0.7446869	total: 2m 11s	remaining: 20.6s
865:	learn: 0.7447098	total: 2m 11s	remaining: 20.4s
866:	learn: 0.7447248	total: 2m 12s	remaining: 20.3s
867:	learn: 0.7447531	total: 2m 12s	remaining: 20.1s
868:	learn: 0.7447777	total: 2m 12s	remaining: 20s
869:	learn: 0.7447905	total: 2m 12s	remaining: 19.8s
870:	learn: 0.7448150	total: 2m 12s	remaining: 19.7s
871:	learn: 0.7448501	total: 2m 12s	remaining: 19.5s
872:	learn: 0.7448783	total: 2m 13s	remaining: 19.4s
873:	learn: 0.7449319	total: 2m 13s	remaining: 19.2s
874:	learn: 0.7449646	total: 2m 13s	remaining: 19s
875:	learn: 0.7449929	total: 2m 13s	remaining: 18.9s
876:	learn: 0.7450112	total: 2m 13s	remaining: 18.7s
877:	learn: 0.7450228	total: 2m 13s	remaining: 18.6s
878:	learn: 0.7450411	total: 2m 13s	remaining: 18.4s
879:	learn: 0.7450646	total: 2m 14s	remaining: 18.3s
880:	learn: 0.7450945	total: 2m 14s	remaining: 18.1s
881:	learn: 0.7451222	total: 2m 14s	remaining: 18s
882:	learn: 0.7451362	total: 2m 14s	remaining: 17.8s

In [94]:
# vclf.fit(X_train_new, Y_train)

In [59]:
Preds = pd.DataFrame(data={'proba': vclf.predict_proba(X_test_new).ravel()}, index=X_test_new.index)

ValueError: Shape of passed values is (1, 200000), indices imply (1, 100000)

In [67]:
np.unique(vclf.predict_proba(X_test_new).ravel())

array([2.02577163e-04, 4.51807171e-04, 9.99548193e-01, 9.99797423e-01])

In [62]:
X_test_new.shape

(100000, 20)

In [70]:
xttrain, xttest, yttrain, yttest = train_test_split(X_train_new, Y_train, test_size=0.2, random_state=13)

In [71]:
clf = LogisticRegression(penalty='l2',C=0.1)
clf.fit(xttrain, yttrain)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [73]:
clf.score(xttest, yttest)

0.8279875657917977

In [74]:
clf.predict_proba(X_test_new)

ValueError: X has 20 features per sample; expecting 22

In [75]:
X_test_new.columns

Index(['Код_диагноза', 'Возраст', 'Пол', 'tfidf_nb_eap_tfidfAnamnes',
       'tfidf_nb_eap_tfidfDiagnoz', 'tfidf_nb_eap_tfidfGeneral',
       'tfidf_nb_eap_tfidfAllergy', 'tfidf_nb_eap_tfidfOutside',
       'tfidf_char_nb_eap_Диагноз', 'tfidf_char_nb_eap_Общее состояние',
       'tfidf_char_nb_eap_аллергия', 'tfidf_char_nb_eap_Анамнез заболевания',
       'tfidf_char_nb_eap_Внешний осмотр', 'count_words_nb_eap_Диагноз',
       'count_words_nb_eap_Общее состояние', 'count_words_nb_eap_аллергия',
       'count_words_nb_eap_Анамнез заболевания',
       'count_words_nb_eap_Внешний осмотр', 'Код_диагноза_0',
       'Код_диагноза_1'],
      dtype='object')

In [79]:
[e for e in X_train_new.columns if e not in X_test_new.columns]

['ID', 'Revisit']